# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [11]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('/Users/zaid/Desktop/Ipython/DataScience-Python3/emails', 'spam'))
data = data.append(dataFrameFromDirectory('/Users/zaid/Desktop/Ipython/DataScience-Python3/emails', 'ham'))


Let's have a look at that DataFrame:

In [12]:
data.head()

class  \
/Users/zaid/Desktop/Ipython/DataScience-Python3...  spam   
/Users/zaid/Desktop/Ipython/DataScience-Python3...  spam   
/Users/zaid/Desktop/Ipython/DataScience-Python3...  spam   
/Users/zaid/Desktop/Ipython/DataScience-Python3...  spam   
/Users/zaid/Desktop/Ipython/DataScience-Python3...  spam   

                                                                                              message  
/Users/zaid/Desktop/Ipython/DataScience-Python3...                                                     
/Users/zaid/Desktop/Ipython/DataScience-Python3...      Date:        Wed, 21 Aug 2002 10:54:46 -05...  
/Users/zaid/Desktop/Ipython/DataScience-Python3...  Martin A posted:\n\nTassos Papadopoulos, the G...  
/Users/zaid/Desktop/Ipython/DataScience-Python3...  Man Threatens Explosion In Moscow \n\n\n\nThur...  
/Users/zaid/Desktop/Ipython/DataScience-Python3...  Klez: The Virus That Won't Die\n\n \n\nAlready...

Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [13]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Let's try it out:

In [14]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['ham', 'ham'], dtype='<U4')